# Conses

a cons is a pair of pointers
- the first one is the car
- the second is the cdr

In [1]:
(setf x (cons 'a nil))

(car x)
(cdr x)

(A)

A

NIL

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X


In [3]:
(setf y (list 'a 'b 'c))
(cdr y)

(A B C)

(B C)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::Y
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::Y


In [4]:
(setf z (list 'a (list 'b 'c) 'd))
(car (cdr z))

(A (B C) D)

(B C)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::Z
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::Z


In [5]:
(defun our-listp (x)
  (or (null x) (consp x)))

(defun out-atom (x)
  (not (consp x)))

OUR-LISTP

OUT-ATOM

# Equality

In [7]:
; eql: check whether same object
(eql (cons 'a nil) (cons 'a nil))

(let ((x (cons 'a nil)))
  (eql x x))

NIL

T

In [8]:
; equal: check whether print the same ???
(let ((x (cons 'a nil)))
  (equal x (cons 'a nil)))

T

In [9]:
(defun our-equal-r (x y)
  (or (eql x y)
      (and (consp x)
           (consp y)
           (our-equal-r (car x) (car y))
           (our-equal-r (cdr x) (cdr y)))))

OUR-EQUAL-R

# Why Lisp Has No Pointers 

In [11]:
(let ((x '(a b c))
      (y nil))
  (setf y x)
  (eql x y))

T

# Building Lists

In [12]:
; copy-list: return a copy of list
(let ((x '(a b c))
      (y nil))
  (setf y (copy-list x)))

(A B C)

In [13]:
(defun our-copy-list-r (lst)
  (if (atom lst)
      lst
      (cons (car lst) (our-copy-list-r (cdr lst)))))

OUR-COPY-LIST-R

In [14]:
; append: return the concatenation of any number of lists
(append '(a b) '(c d) '(e))

(A B C D E)

# Example: Compression

In [ ]:
; run-length encoding

;;; return (n elt) or elt
(defun n-elts (elt n)
  (if (> n 1)
      (list n elt)
      elt))

;;; encoding when elt has occurred n times
(defun compr (elt n lst)
  (if (null lst)
      (list (n-elts elt n))
      (let ((next (car lst))) ; take next element in lst
        (if (eql next elt)
            (compr elt (+ n 1) (cdr lst))
            (cons (n-elts elt n)
                  (compr next 1 (cdr lst)))))))

;;; encoding
(defun compress (x)
  (if (consp x)
      (compr (car x) 1 (cdr x)) ; first occurrence
      x))



;;; return list with length n and all element elt 
(defun list-of (n elt)
  (if (zerop n)
      nil
      (cons elt (list-of (- n 1) elt))))

;;; decoding
(defun uncompress (lst)
  (if (null lst)
      nil
      ;; lst non-null here
      (let ((elt (car lst))
            (rest (uncompress (cdr lst))))
        (if (consp elt) ; elt is cons
            (append (apply #'list-of elt)
              rest)
            (cons elt rest)))))

N-ELTS

COMPR

COMPRESS

LIST-OF

UNCOMPRESS

In [8]:
(n-elts 'a 0)
(n-elts 'a 1)
(n-elts 'a 2)

(list-of 3 'ho)

A

A

(2 A)

(HO HO HO)

In [5]:
(let* ((lst '(1 1 1 0 1 0 0 0 0 1))
  (c (compress lst)))
  (format t "~A: ~A~%" lst c)
  (equal lst (uncompress c)))


T

(1 1 1 0 1 0 0 0 0 1): ((3 1) 0 1 (4 0) 1)


# Access

In [12]:
; nth element
(nth 0 '(a b c))
; nth cdr
(nthcdr 2 '(a b c))

(defun our-nthcdr-r (n lst)
  (if (zerop n)
      lst
      (our-nthcdr-r (- n 1) (cdr lst))))
(our-nthcdr-r 2 '(a b c))


; last cons in list
(last '(a b c))


; others
(first '(a b c))
(second '(a b c))
(tenth '(a b c))

(let ((x '(a b c d)))
  (eql (caddr x) (car (cdr (cdr x)))))

A

(C)

OUR-NTHCDR-R

(C)

(C)

A

B

NIL

T

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::OUR-NTHCDR-R in DEFUN


# Mapping Functions

In [13]:
; mapcar
(mapcar #'(lambda (x) (+ x 10))
        '(1 2 3))

(mapcar #'list
        '(a b c)
        '(1 2 3 4))

(11 12 13)

((A 1) (B 2) (C 3))

In [14]:
; maplist: on successive cdrs of the lists
(maplist #'(lambda (x) x)
         '(a b c))

((A B C) (B C) (C))

In [15]:
; others
; mapc: P.88
; mapcan: P.202

# Trees

In [16]:
(defun our-copy-tree-r (tr)
  (if (atom tr)
      tr
      (cons (our-copy-tree-r (car tr))
            (our-copy-tree-r (cdr tr)))))

(let ((tr '(a '(b c) d)))
  (equal (copy-tree tr) (our-copy-tree-r tr)))

OUR-COPY-TREE-R

T

In [18]:
; substitute: replace element in a sequence
(let ((lst '(and (integerp x) (zerop (mod x 2)))))
  (substitute 'y 'x lst))

; subst: replace elements in a tree
(let ((lst '(and (integerp x) (zerop (mod x 2)))))
  (subst 'y 'x lst))

(AND (INTEGERP X) (ZEROP (MOD X 2)))

(AND (INTEGERP Y) (ZEROP (MOD Y 2)))

In [ ]:
(defun our-subst-r (new old tree)
  (if (eql tree old); atom case
      new
      (if (atom tree)
          tree
          (cons (our-subst-r new old (car tree))
                (our-subst-r new old (cdr tree))))))

(let ((lst '(and (integerp x) (zerop (mod x 2)))))
  (our-subst-r 'y 'x lst))

OUR-SUBST-R

(AND (INTEGERP Y) (ZEROP (MOD Y 2)))

# Understanding Recursion

In [22]:
(defun len (lst)
  (if (null lst)
      0
      (+ (len (cdr lst)) 1)))

(trace len)
(len '(a b c d))
(untrace len)

LEN

(LEN)

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::LEN in DEFUN
SIMPLE-WARNING: LEN is already TRACE'd, untracing it first.
  0: (LEN (A B C D))
    1: (LEN (B C D))
      2: (LEN (C D))
        3: (LEN (D))
          4: (LEN NIL)
          4: LEN returned 0
        3: LEN returned 1
      2: LEN returned 2
    1: LEN returned 3
  0: LEN returned 4


4

T

# Sets

In [27]:
; member: return the part of the list beginning with lookup object
(member 'b '(a b c))
; :test
(member '(a) '((a) (z))) ; defualt test with eql
(member '(a) '((a) (z)) :test #'equal)
; :key
(member 'a '((a b) (c d)) :key #'car)

(member 2 '((1) (2)) :key #'car :test #'equal)

(B C)

NIL

((A) (Z))

((A B) (C D))

((2))

In [28]:
(member-if #'oddp '(2 3 4))

(defun our-member-if-r (fn lst)
  (and (consp lst)
       (if (funcall fn (car lst))
           lst
           (our-member-if-r fn (cdr lst)))))
(our-member-if-r #'oddp '(2 3 4))

(3 4)

OUR-MEMBER-IF-R

(3 4)

In [29]:
; adjoin: cons the object onto the list only if it is not a meber
(adjoin 'b '(a b c))
(adjoin 'z '(a b c))

(A B C)

(Z A B C)

In [30]:
; set operation: union, intersection, difference
(union '(a b c) '(c b s))
(intersection '(a b c) '(b b c))
(set-difference '(a b c d e) '(b e))

(A C B S)

(C B)

(D C A)

# Sequences
- lists
- vectors

In [32]:
(length '(a b c))

(subseq '(a b c d) 1 2)
(subseq '(a b c d) 1)

(reverse '(a b c))

3

(B)

(B C D)

(C B A)

In [49]:
(defun mirror? (s)
  (let ((len (length s)))
    ; (format t "~A ~A ~A~%" len (oddp len) (evenp len))
    (or
     (and (oddp len)
          (let ((mid (ceiling len 2)))
            ; (format t "~A ~A~%" (subseq s 0 (- mid 1)) (subseq s mid))
            (equal (subseq s 0 (- mid 1))
                   (subseq s mid))))
     (and (evenp len)
          (let ((mid (/ len 2)))
            (equal (subseq s 0 mid)
                   (reverse (subseq s mid))))))))

; (mirror? '(a b b a))
(mirror? '(a b a))

MIRROR?

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MIRROR? in DEFUN
  0: (MIRROR? (A B A))
  0: MIRROR? returned T


T

In [44]:
(ceiling 3 2)

2

-1

In [51]:
; sort: destructive
(sort '(0 2 1 3 8) #'>)

(defun nthmost (n lst)
  (nth (- n 1)
       (sort (copy-list lst) #'>)))
(nthmost 2 '(0 2 1 3 8))

(8 3 2 1 0)

NTHMOST

3

SB-INT:CONSTANT-MODIFIED: Destructive function SORT called on constant data: (0 2 1 3 8)
See also:
  The ANSI Standard, Special Operator QUOTE
  The ANSI Standard, Section 3.7.1


In [52]:
; every
(every #'oddp '(1 3 5))
(every #'> '(1 3 5) '(0 2 4))
; some
(some #'evenp '(1 2 3))

T

T

T

# Stacks

In [54]:
; push, pop
(let ((x '(b))
      (y nil))
  (push 'a x)
  (format t "x=~A~%" x)
  (setf y x)
  (pop x)
  (format t "x=~A y=~A~%" x y))

NIL

x=(A B)
x=(B) y=(A B)


In [55]:
(defun our-reverse (lst)
  (let ((acc nil))
    (dolist (elt lst)
      (push elt acc))
    acc))

(let ((l '(a b c)))
  (equal (reverse l) (our-reverse l)))

OUR-REVERSE

T

In [56]:
; pushnew: macro, variant of push use adjoin instead of cons
(let ((x '(a b)))
  (pushnew 'c x)
  (pushnew 'a x)
  x)

(C A B)

# Dotted Lists

In [57]:
; property list: either nil, or a cons whose cdr is a proper list
(defun proper-list? (x)
  (or (null x)
      (and (consp x)
           (proper-list? (cdr x)))))

; dotted list: a cons that is NOT a proper list
(cons 'a 'b)
'(a b)
(proper-list? (cons 'a 'b))
(proper-list? '(a b))

PROPER-LIST?

(A . B)

(A B)

NIL

T

In [58]:
'(a . (b . (c . nil)))
(cons 'a (cons 'b (cons 'c 'd)))

(A B C)

(A B C . D)

In [59]:
; denote list
'(a . (b . nil))
'(a . (b))
'(a b . nil)
'(a b)

(A B)

(A B)

(A B)

(A B)

# Assoc-lists

In [60]:
; assoc-list, alist: a list of conses
(setf trans '((+ . "add") (- . "subtract")))
(assoc '+ trans)
(assoc '* trans)

((+ . "add") (- . "subtract"))

(+ . "add")

NIL

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::TRANS
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::TRANS
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::TRANS


In [ ]:
(defun our-assoc-r (key alist)
  (and (consp alist)
       (let ((pair (car alist)))
         (if (eql key (car pair))
             pair
             (our-assoc-r key (cdr alist))))))

(let ((trans '((+ . "add") (- . "subtract"))))
  (every #'equal (list (assoc '+ trans) (assoc '* trans))
    (list (our-assoc-r '+ trans) (our-assoc-r '* trans))))

OUR-ASSOC-R

T

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::OUR-ASSOC-R in DEFUN


# Example: Shortest Path

In [ ]:
(defun shorteset-path (start end net)
  (bfs end (list (list start)) net)) ; queue: ((start))

(defun bfs (end queue net)
  (format t "Queue: ~A~%" queue)
  (if (null queue)
      nil
      (let ((path (car queue))) ; first path in queue
        (let ((node (car path))) ; first node in path
          (if (eql node end)
              (progn 
                ; (format t "Found ~A in ~A~%" node path)
                (reverse path))
              (bfs end
                   ;; remove this path, append paths reachable from node
                   (append (cdr queue) (new-paths path node net))
                   net))))))


(defun new-paths (path node net)
  (format t "  Path: ~A, node: ~A~%" path node)
  (mapcar #'(lambda (n) (cons n path)) ;; prepend n to path
    ;; node's neighbors
    (cdr (assoc node net))))


SHORTESET-PATH

BFS

NEW-PATHS

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SHORTESET-PATH in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::BFS in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::NEW-PATHS in DEFUN


In [68]:
(let ((min-net '((a b c) (b c) (c d))))
  (new-paths '() 'a min-net))

((B) (C))

  Path: NIL, node: A


In [81]:
(trace bfs)

; a -> b, a -> c
; b -> c
; c -> d
(let ((min-net '((a b c) (b c) (c d))))
  (shorteset-path 'a 'd min-net))

(untrace bfs)

(BFS)

  0: (BFS D ((A)) ((A B C) (B C) (C D)))
Queue: ((A))
  Path: (A), node: A
    1: (BFS D ((B A) (C A)) ((A B C) (B C) (C D)))
Queue: ((B A) (C A))
  Path: (B A), node: B
      2: (BFS D ((C A) (C B A)) ((A B C) (B C) (C D)))
Queue: ((C A) (C B A))
  Path: (C A), node: C
        3: (BFS D ((C B A) (D C A)) ((A B C) (B C) (C D)))
Queue: ((C B A) (D C A))
  Path: (C B A), node: C
          4: (BFS D ((D C A) (D C B A)) ((A B C) (B C) (C D)))
Queue: ((D C A) (D C B A))
          4: BFS returned (A C D)
        3: BFS returned (A C D)
      2: BFS returned (A C D)
    1: BFS returned (A C D)
  0: BFS returned (A C D)


(A C D)

T

In [84]:
;
; analysis
;
; ((A)); a: a->b, a->c
; ((B A) (C A)) ; b: b->c
; ((C A) (C B A)) ; c: c -> d
; ((C B A) (D C A)); c: c -> d
; ((D C A) (D C B A)); d

# Garbage

In [85]:
; where does garbage come from?
(setf lst (list 'a 'b 'c))
(setf lst nil)

(A B C)

NIL

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::LST
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::LST
